In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input, GRU
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from models_def import Attention

max_features = 180000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)


tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
# 
word_vec_dict = {}
with open('../crawl-300d-2M.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.rstrip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')

EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

Using TensorFlow backend.


(159571, 6)
(159571, 150) (153164, 150)
engrams 81151
oshea 129810
costarred 66086
xr6 155121
bogdanovs 21354
teilman 133131
2000000
Preparing embedding matrix
Null word embeddings: 79399


In [6]:
from sklearn.metrics import log_loss,accuracy_score
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU,CuDNNGRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
import gc


def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def get_model(comp=True):
    inp = Input(shape=(maxlen, ))
    x = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inp)
    x = SpatialDropout1D(0.4)(x)
    gru_out = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    
    x = Conv1D(64, kernel_size = 2, padding = "valid")(gru_out)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    att = Attention(maxlen)(gru_out)
    conc = concatenate([att,avg_pool, max_pool])
    
    # dr and BN
    conc = Dropout(0.2)(conc)
    conc = BatchNormalization()(conc)
    outp = Dense(6, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    if comp:
        model.compile(loss='binary_crossentropy',
                      optimizer='nadam',
                      metrics=['accuracy'])

    return model
print('def model done')

def model done


In [7]:
m = get_model(False)
m.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 150, 300)     54000000    input_2[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 150, 300)     0           embedding_2[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 150, 256)     330240      spatial_dropout1d_2[0][0]        
__________________________________________________________________________________________________
conv1d_2 (

In [8]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_model()
        batch_size = 64
        epochs = 8
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, 
                  batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), 
                  callbacks=callbacks_list)
        
        model.load_weights(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        
        # clear
#         del model
#         gc.collect()
#         K.clear_session()
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


print('def done')

def done


In [9]:
import pickle
sample_submission = pd.read_csv("../input/sample_submission.csv")

train_pred,test_pred = kf_train(fold_cnt=4,rnd=4)
print(train_pred.shape,test_pred.shape)    

sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/pool_gru1_fasttext_adj2_sample_4.gz", index=False, compression='gzip')
with open('../features/pool_gru_fasttext_adj2_4_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print(sample_submission.head())
print('===================================')

# fold10 test
# check fisrt 1 fold
# gru  conv val_loss
# pre       3865 3960
# 128  128  3954 
# 128  64   3901
# 96   64   3928

# dr    val_loss
# 0.4   3881
# 0.3   3976
# 0.45  3988

# add dr and BN at bottom
# bottom dr  val_loss
# 0.3        3900
# 0.2 nadam  3865 4090 seems good
# 0.1 nadam  3921

# fold4 output feat, fail?

Train on 119678 samples, validate on 39893 samples
Epoch 1/8
119678/119678 [==============================] - 100s 836us/step - loss: 0.0725 - acc: 0.9743 - val_loss: 0.0446 - val_acc: 0.9830
Epoch 2/8
119678/119678 [==============================] - 105s 875us/step - loss: 0.0496 - acc: 0.9817 - val_loss: 0.0417 - val_acc: 0.9839
Epoch 3/8
119678/119678 [==============================] - 107s 890us/step - loss: 0.0450 - acc: 0.9832 - val_loss: 0.0412 - val_acc: 0.9838
Epoch 4/8
119678/119678 [==============================] - 105s 875us/step - loss: 0.0425 - acc: 0.9837 - val_loss: 0.0403 - val_acc: 0.9841
Epoch 5/8
119678/119678 [==============================] - 105s 880us/step - loss: 0.0414 - acc: 0.9842 - val_loss: 0.0399 - val_acc: 0.9842
Epoch 6/8
119678/119678 [==============================] - 106s 886us/step - loss: 0.0392 - acc: 0.9848 - val_loss: 0.0411 - val_acc: 0.9835
Epoch 7/8
119678/119678 [==============================] - 105s 878us/step - loss: 0.0379 - acc: 0.9853

InternalError: Failed to call ThenRnnForward
	 [[Node: bidirectional_1/CudnnRNN = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="gru", seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/gpu:0"](bidirectional_1/transpose, bidirectional_1/ExpandDims_1, bidirectional_1/Const, bidirectional_1/concat)]]
	 [[Node: loss/mul/_177 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2185_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'bidirectional_1/CudnnRNN', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <mod